# Supermarkets modeling

The goal of this document....

In [14]:
import pandas as pd
import numpy as np

## 1. Import data

In [15]:
# import data
supermarkets = {
    'aldinorth': pd.read_pickle('data/products-clean/aldinorth-products-clean.pkl'),
    'aldisouth': pd.read_pickle('data/products-clean/aldisouth-products-clean.pkl'),
    'edeka': pd.read_pickle('data/products-clean/edeka-products-clean.pkl')
    #'kaufan': pd.read_pickle('data/products-clean/edeka-products-clean.pkl')
    #'rewe': pd.read_pickle('data/products-clean/edeka-products-clean.pkl')
    #'lidl': pd.read_pickle('data/products-clean/edeka-products-clean.pkl')
}

In [17]:
supermarkets['aldinorth'].head()

,Name,Price
0,Tamara extra strawberry jam,0.99
1,Nutella,3.79
2,Coke,2.89
3,Trader Joe's / Aldi Nord Lemon Iced Tea,0.49
4,Iglo fish fingers,3.49


## 2. Modeling to do's

- Algorithm 1.0 - Build an algorithm able to (input: ingredient, output: % coverage + total price per each supermarket)
- Algorithm 2.0 - Build an algorithm able to (input: list of ingredients, output: % coverage + total price per each supermarket)
- Algorithm 3.0 - Use fuzzy buzzy to match the input (potato) with the ingredients by similarity.

### 2.1 Algorithm 1.0
Build an algorithm able to: 

- [input]: ingredient

- [output]: price in each supermarket

In [18]:
# We are looking for the price of the ingredient that we are searching per each supermarket
supermarkets['aldinorth'][supermarkets['aldinorth']['Name'] == 'Nutella']['Price'].values[0]

3.79

In [27]:
# Function to get the prices with an ingredient as input
def search(ingredient:str):
    # New empty list
    result = []
    for market in supermarkets:
        a = supermarkets[market]
        try:
            price = a[a['Name'] == ingredient]['Price'].values[0]
            result.append(f'{market}: {price}')
        except:
            result.append(f'{market}: Not found')
    return result

# Try it with 'Nutella' because it is available in every supermarket
# Try with 'Coke'

In [28]:
# Testing the function
search('Nutella')

['aldinorth: 3.79', 'aldisouth: 3.89', 'edeka: 1.77']

In [29]:
# Testing the function
search('Coke')

['aldinorth: 2.89', 'aldisouth: 0.99', 'edeka: 1.39']

We can see that there are some inconsistencies with the 'Coca Cola' prices. The problem is that the size of the packs are not normalized. 

In [31]:
supermarkets['edeka'][supermarkets['edeka']['Name'] == 'Coke']

,Name,Price
0,Coke,1.39


In [39]:
supermarkets['edeka'][supermarkets['edeka']['Name'].str.contains('Coke')]

,Name,Price
0,Coke,1.39


### 2.2 Algorithm 2.0
Build an algorithm able to:

- [input]: list of ingredients (ing as it is in the dataset)
- [output]: % coverage + total price per each supermarket

In [40]:
def search_list(ingredients:list):
    # New empty list
    result = []
    
    for market in supermarkets:
        
        a = supermarkets[market]
        
        for ing in ingredients:    
            try:
                price = a[a['Name'] == ing]['Price'].values[0]
                result.append(f'{market}: {price} for the {ing}')
            except:
                result.append(f'{market}: Not found')
            
    return result

In [42]:
search_list(['Coke', 'Nutella'])

['aldinorth: 2.89 for the Coke',
 'aldinorth: 3.79 for the Nutella',
 'aldisouth: 0.99 for the Coke',
 'aldisouth: 3.89 for the Nutella',
 'edeka: 1.39 for the Coke',
 'edeka: 1.77 for the Nutella']

In [153]:
result = pd.DataFrame(columns=['Supermarket', 'Ingredient', 'Price', 'Coverage'])

In [154]:
result.append({'Supermarket': 'aldinorth', 'Ingredient': 'Coke', 'Price': 2.89, 'Coverage': 1}, ignore_index=True)

,Supermarket,Ingredient,Price,Coverage
0,aldinorth,Coke,2.89,1


In [157]:
result['Price'] = pd.to_numeric(result['Price'], errors='coerce')
result['Coverage'] = pd.to_numeric(result['Coverage'], errors='coerce')

In [158]:
result.dtypes

Supermarket    object
Ingredient     object
Price           int64
Coverage        int64
dtype: object

In [188]:
def search_list_grouped(ingredients:list):
    # New empty df
    result = pd.DataFrame(columns=['Supermarket', 'Ingredient', 'Price', 'Coverage'])

    for market in supermarkets:
        
        a = supermarkets[market]
        
        for ing in ingredients:    
            try:
                price = a[a['Name'] == ing]['Price'].values[0]
                df2 = {'Supermarket': market, 'Ingredient': ing, 'Price': price, 'Coverage': 1}
                result = result.append(df2, ignore_index=True)
            except:
                result = result.append({'Supermarket': market, 'Ingredient': ing, 'Price': np.nan, 'Coverage': 0}, ignore_index=True)
    
    # Change to numeric
    result['Price'] = pd.to_numeric(result['Price'], errors='coerce')
    result['Coverage'] = pd.to_numeric(result['Coverage'], errors='coerce') * 100
    
    # Group by 'Supermarket'
    results = result.groupby('Supermarket').agg({'Price':'sum', 'Coverage': 'mean'}).reset_index()
    results = results.sort_values('Price').reset_index(drop=True)
    
    # How to display it to users
#    for row in range(len(results)):
#        print(f"In {row['Supermarket']} you can find {row['Coverage']}% of the ingredients for row['Price']")
    
    return results

# 'result' dataframe contains detailes information
# 'results' dataframe contains grouped information

In [189]:
search_list_grouped(['Coke', 'Nutella', 'Vanilla', 'Tuna', 'Soap', 'Classic Tomato Sauce'])

,Supermarket,Price,Coverage
0,edeka,3.16,33.333333
1,aldisouth,4.88,33.333333
2,aldinorth,6.68,33.333333


As we can see, if we do not type the ingredient exactly as it is in the dataset, our algorithm is not able to find it. In order to make it more useful, we are going to add fuzzy words and create a basic search engine.

### 2.3 Algorithm 3.0

Build an algorithm able to:

- [input]: list of ingredients (similar word)
- [output]: % coverage + total price per each supermarket

In [ ]:
# Add the packages

